# Numerics of Machine Learning
# Exercise Sheet No. 11 — Optimization for Deep Learning

---
University of Tübingen, Winter Term 2022/23
&copy; N. Bosch, J. Grosse, P. Hennig, A. Kristiadi, M. Pförtner, J. Schmidt, F. Schneider, L. Tatzel, J. Wenger, 2022 CC BY-NC-SA 3.0

In [ ]:
from torch import nn, optim
from torch.utils.data import dataloader, random_split
from torchvision import datasets, transforms

from backpack import extend
from cockpit import Cockpit, CockpitPlotter, quantities
from cockpit.utils.configuration import configuration, schedules

from utils import (
    get_default_device,
    accuracy_fn,
    eval_model,
    visualize_results,
    set_seeds,
    get_logpath,
)

## Loading SVHN Data

In [ ]:
def build_dataloaders(batch_size, verbose=False):
    """Download the data and build dataloaders."""

    # Provide verbose output
    if verbose:
        print("** Loading Data **")

    # Basic transformation of the data
    basic_transform = transforms.Compose(
        [
            transforms.RandomRotation(degrees=(0, 180)),
            transforms.ToTensor(),
        ]
    )

    # Load datasets
    train_data = datasets.SVHN(
        root="data/", split="train", transform=basic_transform, download=True
    )
    eval_data = datasets.SVHN(
        root="data/", split="train", transform=basic_transform, download=True
    )

    # Use half of the eval_data for the validation set and half for the test set
    split = int(len(eval_data) / 2)
    valid_data, test_data = random_split(eval_data, [split, len(eval_data) - split])

    # Build dataloaders
    train_loader = dataloader.DataLoader(train_data, batch_size, shuffle=True)
    val_loader = dataloader.DataLoader(valid_data, batch_size)
    test_loader = dataloader.DataLoader(test_data, batch_size)

    # Create dict with names
    dataloaders = {
        "train": train_loader,
        "validation": val_loader,
        "test": test_loader,
    }

    return dataloaders

## Defining the Model

In [ ]:
def CNN(verbose=False):
    """Basic Conv-Net for SVHN."""
    if verbose:
        print("** Creating Model **")

    return nn.Sequential(
        nn.Conv2d(3, 32, 3, padding=1),
        nn.Tanh(),
        nn.Conv2d(32, 32, 3, padding=1),
        nn.Tanh(),
        nn.MaxPool2d(2),
        nn.Conv2d(32, 64, 3, padding=1),
        nn.Tanh(),
        nn.Conv2d(64, 64, 3, padding=1),
        nn.Tanh(),
        nn.MaxPool2d(2),
        nn.Conv2d(64, 128, 3, padding=1),
        nn.Tanh(),
        nn.Conv2d(128, 128, 3, padding=1),
        nn.Tanh(),
        nn.MaxPool2d(2),
        nn.Flatten(),
        nn.Linear(2048, 128),
        nn.Tanh(),
        nn.Linear(128, 10),
    )

# Train Loop

In [ ]:
# Set device for training (use GPU if possible)
DEVICE = get_default_device()


def train(num_epochs=10, batch_size=256, lr=1e-5, verbose=False, cockpit=False):
    """Train the model."""

    # Set random seeds
    set_seeds()

    # Data
    dataloaders = build_dataloaders(batch_size, verbose)

    # Model
    model = CNN(verbose=verbose)
    if cockpit:
        model = extend(model)
    model.to(DEVICE)

    # Loss
    loss_fn = nn.CrossEntropyLoss(reduction="mean")
    individual_loss_fn = nn.CrossEntropyLoss(reduction="none")
    if cockpit:
        loss_fn = extend(loss_fn)
        individual_loss_fn = extend(individual_loss_fn)

    # Eval
    eval_fn = accuracy_fn
    results = {
        "train": {"loss": [], "accuracy": []},
        "validation": {"loss": [], "accuracy": []},
        "test": {"loss": [], "accuracy": []},
    }

    # Cockpit
    if cockpit:
        quants = [
            quantities.Alpha(schedules.linear(interval=10)),
            quantities.Distance(schedules.linear(interval=1)),
            quantities.GradNorm(schedules.linear(interval=1)),
            quantities.Loss(schedules.linear(interval=1)),
            quantities.UpdateSize(schedules.linear(interval=1)),
        ]
        cockpit = Cockpit(
            model.parameters(),
            quantities=quants,
        )

    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)
    lr_schedule = optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=1.001)

    # Train Loop
    global_step = 0
    for epoch in range(num_epochs):
        if verbose:
            print(f"** Starting Epoch {epoch} on {DEVICE}**")

        for batch_iter, batch in enumerate(dataloaders["train"]):
            # Get inputs and labels
            inputs, targets = batch[0].to(DEVICE), batch[1].to(DEVICE)

            # Forward pass
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            losses = individual_loss_fn(outputs, targets)

            # Backward pass
            if cockpit:
                with cockpit(
                    global_step,
                    info={
                        "batch_size": inputs.shape[0],
                        "individual_losses": losses,
                        "loss": loss,
                        "optimizer": optimizer,
                    },
                ):
                    loss.backward(create_graph=cockpit.create_graph(global_step))
            else:
                loss.backward()

            # Update step
            optimizer.step()
            global_step += 1
            lr_schedule.step()

        # Per-epoch eval
        results = eval_model(
            model, epoch, results, DEVICE, loss_fn, eval_fn, dataloaders, verbose
        )

        # Cockpit Log
        if cockpit:
            cockpit.log(
                global_step,
                epoch,
                results["train"]["loss"][-1],
                results["validation"]["loss"][-1],
                results["test"]["loss"][-1],
                results["train"]["accuracy"][-1],
                results["validation"]["accuracy"][-1],
                results["test"]["accuracy"][-1],
                optimizer.param_groups[0]["lr"],
            )

    # Write Cockpit to json file.
    if cockpit:
        cockpit.write(get_logpath())

    return results, cockpit

## Train the Model and Show Results

In [ ]:
%%time

verbose = True

results, cockpit = train(num_epochs=10, verbose=verbose, cockpit=True)
visualize_results(results, verbose)

## Show Cockpit

In [ ]:
# Plot results from latest cockpit logfile
# Requires cockpit=True in the train method
plotter = CockpitPlotter()
plotter.plot(get_logpath())